В этом блокноте рассмотрены бейзлайны Top popular и Top Personal

In [4]:
from google.colab import drive
drive.mount('/content/drive')

#%cd /content/drive/MyDrive/recsys

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir("/content/drive/MyDrive/recsys")
!ls

checkpoints  metrics.py  __pycache__	      recanet_model.py
data	     preprocess  recanet_datasets.py


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.2)

from IPython.display import clear_output
%matplotlib inline

import pandas as pd
import plotly.express as px
import numpy as np
from tqdm import tqdm

tqdm.pandas()

from sklearn.metrics import accuracy_score, roc_auc_score

from metrics import recall_k,precision_k, hitrate_k, ndcg_k, repeat_score_item, repeat_score_user
from recanet.recanet_datasets import PreDataset, RCNDataset, DeviceDataLoader

from metrics import recall_k, ndcg_k, repeat_score_item, repeat_score_user
from boosting.create_dataset import Dataset


In [6]:
def top_pop(train, test_users):
    result = {}
    top_popular = train.item_id.value_counts().index.tolist()
    train = train.sort_values(by='date')
    for user in tqdm(test_users):
        result[user] = top_popular
        
    return result

In [7]:
def top_person(train, test_users):
    result = {}
    train = train.sort_values(by='date')
    
    for user in tqdm(test_users):
        items = train[train.user_id==user]
        top_personal = items.item_id.value_counts().index.tolist()
        result[user] = top_personal
        
    return result

In [8]:
def calculate_metrics(path_test, test_users, res):
    test_baskets = pd.read_csv(path_test)
    user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
    user_test_baskets_dict = dict(zip(user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

    user_predictions = res
    final_users = test_users
    print('predictions ready',len(user_predictions))
    print('number of final test users:',len(final_users))
    for k in [5,10,20,'B']:
        print(k)
        recall_scores = {}
        ndcg_scores = {}
        precision_scores = {}
        hitrate_scores = {}
        zero = 0
        for user in final_users:

            top_items = []
            if user in user_predictions:
                top_items = user_predictions[user]
            else:
                zero+=1

            if k == 'B':
                recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
                ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
                precision_scores[user] = precision_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
                hitrate_scores[user] = hitrate_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
            else:
                recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
                ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
                precision_scores[user] = precision_k(user_test_baskets_dict[user],top_items,k)
                hitrate_scores[user] = hitrate_k(user_test_baskets_dict[user],top_items,k)
        #print(zero)
        print('recall:',np.mean(list(recall_scores.values())))
        print('ndcg:',np.mean(list(ndcg_scores.values())))
        print('precision:',np.mean(list(precision_scores.values())))
        print('hitrate:',np.mean(list(hitrate_scores.values())))


## Ta-Feng

In [2]:
dataset_name = 'tafeng'

In [5]:
path_train = f'data/{dataset_name}/baskets/train_baskets.csv'
path_test = f'data/{dataset_name}/baskets/test_baskets.csv'
path_val = f'data/{dataset_name}/baskets/valid_baskets.csv'

In [6]:
dataset = Dataset(path_train,path_val, path_test, dataset=dataset_name, history_len=50, basket_len=20)

Total users: 10205
Total items: 13521


In [7]:
train = pd.read_csv(path_train)
test = dataset.test_cleaned
test_users = np.intersect1d(train.user_id.unique(), test.user_id.unique(), assume_unique=True)

In [21]:
len(test_users)

5007

### Top popular

In [23]:
res = top_pop(train, test_users)

100%|██████████████████████████████████| 5007/5007 [00:00<00:00, 2671867.70it/s]


In [27]:
calculate_metrics(path_test, test_users, res)

predictions ready 5007
number of final test users: 5007
5
recall: 0.06932809448375854
ndcg: 0.0667396039248835
precision: 0.04801278210505293
hitrate: 0.21070501298182545
10
recall: 0.0803614198488835
ndcg: 0.048200582947222036
precision: 0.03089674455761933
hitrate: 0.2590373477132015
20
recall: 0.1085616074831435
ndcg: 0.03521215691537222
precision: 0.0213600958657879
hitrate: 0.3373277411623727
B
recall: 0.06523485242251235
ndcg: 0.07956851264869756
precision: 0.06523485242251235
hitrate: 0.24405831835430397


### Top personal

In [32]:
res = top_person(train, test_users)

100%|█████████████████████████████████████| 5007/5007 [00:02<00:00, 2304.18it/s]


In [33]:
calculate_metrics(path_test, test_users, res)

predictions ready 5007
number of final test users: 5007
5
recall: 0.07940832980618531
ndcg: 0.08502510232554195
precision: 0.074415817855003
hitrate: 0.28080687038146596
10
recall: 0.10956279446942792
ndcg: 0.06727576025325029
precision: 0.05430397443578989
hitrate: 0.3664869183143599
20
recall: 0.14488960390695454
ndcg: 0.05115722617325702
precision: 0.03800679049330937
hitrate: 0.4519672458558019
B
recall: 0.06954457276485522
ndcg: 0.07992055958958502
precision: 0.06954457276485522
hitrate: 0.30197723187537445


## Dunnhumby

In [34]:
dataset_name = 'dunnhumby_cj'

In [35]:
path_train = f'data/{dataset_name}/baskets/train_baskets.csv'
path_test = f'data/{dataset_name}/baskets/test_baskets.csv'
path_val = f'data/{dataset_name}/baskets/valid_baskets.csv'

In [36]:
dataset = Dataset(path_train,path_val, path_test, dataset=dataset_name, history_len=50, basket_len=20)

Total users: 2483
Total items: 36963


In [37]:
train = pd.read_csv(path_train)
test = dataset.test_cleaned
test_users = np.intersect1d(train.user_id.unique(), test.user_id.unique(), assume_unique=True)

In [38]:
len(test_users)

1224

### Top popular

In [39]:
res = top_pop(train, test_users)

100%|██████████████████████████████████| 1224/1224 [00:00<00:00, 1101679.85it/s]


In [40]:
calculate_metrics(path_test, test_users, res)

predictions ready 1224
number of final test users: 1224
5
recall: 0.060090665960795396
ndcg: 0.0819196487859568
precision: 0.06911764705882352
hitrate: 0.29820261437908496
10
recall: 0.07076300894973854
ndcg: 0.06448491548320197
precision: 0.050490196078431374
hitrate: 0.3570261437908497
20
recall: 0.0855431706906435
ndcg: 0.04833435173703284
precision: 0.03480392156862745
hitrate: 0.4125816993464052
B
recall: 0.039262437175654884
ndcg: 0.049106318996629456
precision: 0.039262437175654884
hitrate: 0.3423202614379085


### Top personal

In [41]:
res = top_person(train, test_users)

100%|██████████████████████████████████████| 1224/1224 [00:01<00:00, 941.93it/s]


In [42]:
calculate_metrics(path_test, test_users, res)

predictions ready 1224
number of final test users: 1224
5
recall: 0.104969895297816
ndcg: 0.16588679158121175
precision: 0.1511437908496732
hitrate: 0.4452614379084967
10
recall: 0.14269457018949996
ndcg: 0.13607489247793672
precision: 0.11576797385620914
hitrate: 0.5277777777777778
20
recall: 0.193195193095419
ndcg: 0.10746619376850698
precision: 0.08553921568627451
hitrate: 0.6053921568627451
B
recall: 0.11237609816445047
ndcg: 0.12998199117086634
precision: 0.11237609816445047
hitrate: 0.4795751633986928


## Instacart

In [43]:
dataset_name = 'instacart'

In [44]:
path_train = f'data/{dataset_name}/baskets/train_baskets_sample30k.csv'
path_test = f'data/{dataset_name}/baskets/test_baskets_sample30k.csv'
path_val = f'data/{dataset_name}/baskets/valid_baskets_sample30k.csv'

In [45]:
dataset = Dataset(path_train,path_val, path_test, dataset=dataset_name, history_len=50, basket_len=20)

Total users: 26828
Total items: 29396


In [46]:
train = pd.read_csv(path_train)
test = dataset.test_cleaned
test_users = np.intersect1d(train.user_id.unique(), test.user_id.unique(), assume_unique=True)

In [47]:
len(test_users)

13246

### Top popular

In [48]:
res = top_pop(train, test_users)

100%|████████████████████████████████| 13246/13246 [00:00<00:00, 3356760.97it/s]


In [49]:
calculate_metrics(path_test, test_users, res)

predictions ready 13246
number of final test users: 13246
5
recall: 0.04543579563427041
ndcg: 0.10459920340413938
precision: 0.09266193567869546
hitrate: 0.35739091046353616
10
recall: 0.06861499976112148
ndcg: 0.08580843786066904
precision: 0.07159897327495093
hitrate: 0.4540238562584931
20
recall: 0.09356497021115388
ndcg: 0.06590336719775221
precision: 0.050502038351200366
hitrate: 0.519477578136796
B
recall: 0.06607454036120189
ndcg: 0.08003729875231046
precision: 0.06607454036120189
hitrate: 0.4441340782122905


### Top personal

In [50]:
res = top_person(train, test_users)

100%|████████████████████████████████████| 13246/13246 [00:20<00:00, 638.28it/s]


In [51]:
calculate_metrics(path_test, test_users, res)

predictions ready 13246
number of final test users: 13246
5
recall: 0.2270130757728264
ndcg: 0.3757888373781584
precision: 0.3456288690925563
hitrate: 0.7750264230711158
10
recall: 0.32444535083048615
ndcg: 0.312163263463353
precision: 0.2693341386078816
hitrate: 0.8536161860184207
20
recall: 0.43183688965264927
ndcg: 0.24446158401490178
precision: 0.19468141325683225
hitrate: 0.9008002415823645
B
recall: 0.29032154701391005
ndcg: 0.32896974413950153
precision: 0.29032154701391005
hitrate: 0.7922391665408425


# Valued shoppers

In [2]:
dataset_name = 'valued_shoppers'

In [14]:
path_train = f'data/{dataset_name}/baskets/train_baskets.csv'
path_test = f'data/{dataset_name}/baskets/test_baskets.csv'
path_val = f'data/{dataset_name}/baskets/valid_baskets.csv'

In [15]:
dataset = Dataset(path_train,path_val, path_test, dataset=dataset_name, history_len=50, basket_len=20)

Total users: 5865
Total items: 12126


In [16]:
train = pd.read_csv(path_train)
test = dataset.test_cleaned
test_users = np.intersect1d(train.user_id.unique(), test.user_id.unique(), assume_unique=True)

In [17]:
len(test_users)

2928

### Top popular

In [18]:
res = top_pop(train, test_users)

100%|█████████████████████████████████████████████████████████████████████████| 2928/2928 [00:00<00:00, 2926119.16it/s]


In [19]:
calculate_metrics(path_test, test_users, res)

predictions ready 2928
number of final test users: 2928
5
recall: 0.05790227545553841
ndcg: 0.10761658413367055
precision: 0.09637978142076502
hitrate: 0.35963114754098363
10
recall: 0.0816685766972876
ndcg: 0.0868473621494549
precision: 0.07226775956284154
hitrate: 0.4504781420765027
20
recall: 0.11922624159285719
ndcg: 0.06856232518218654
precision: 0.053705601092896176
hitrate: 0.557035519125683
B
recall: 0.06946317796936506
ndcg: 0.08283653306866574
precision: 0.06946317796936506
hitrate: 0.42827868852459017


### Top personal

In [20]:
res = top_person(train, test_users)

100%|█████████████████████████████████████████████████████████████████████████████| 2928/2928 [00:17<00:00, 172.20it/s]


In [21]:
calculate_metrics(path_test, test_users, res)

predictions ready 2928
number of final test users: 2928
5
recall: 0.14610950543025483
ndcg: 0.24567081765768287
precision: 0.22349726775956288
hitrate: 0.592896174863388
10
recall: 0.20681950845838382
ndcg: 0.2015245278283286
precision: 0.17127732240437157
hitrate: 0.6844262295081968
20
recall: 0.2876884628501837
ndcg: 0.15964183989263286
precision: 0.12699795081967213
hitrate: 0.762636612021858
B
recall: 0.17671229247894704
ndcg: 0.2047757553843304
precision: 0.17671229247894704
hitrate: 0.6092896174863388
